# Finite volume methods for the Darcy equation

In this tutorial we present how to solve a Darcy equation with [PorePy](https://github.com/pmgbergen/porepy) by using two finite volume methods: two-point flux approximation (TPFA) and multi-point flux approximation (MPFA).

## Exercise 3

We consider the primal formulation of the Darcy problem: the only unknown is the pressure $p$.

Let $\Omega$ be the domain of interest with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $p$ such that
$$
\nabla \cdot (- k\nabla p) = 0
\quad \text{in } \Omega
$$
with boundary conditions:
$$
\left\{
\begin{array}{ll}
p = 10^6 & \text{ on } \partial_{top} \Omega \\
p = 0 & \text{ on } \partial_{bottom} \Omega\\
-\nabla p \cdot \nu = 0  & \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega
\end{array}
\right.
$$
$\Omega$ and the permeability $k$ are taken from the spe10 benchmark case, see [10.2118/66599-MS](http://dx.doi.org/10.2118/66599-MS), by selecting one or multiple layers. In the latter case on the two additional boundary we impose null flux.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

In [ ]:
import numpy as np
import porepy as pp
import scipy.sparse as sps

import os

cwd_folder = os.getcwd()
spe10_folder = cwd_folder + "/spe10/"

import sys

sys.path.insert(1, spe10_folder)

from spe10 import Spe10

We first set some data that will be used in the sequel

In [ ]:
key = "flow"
tol = 1e-6
selected_layers = 35

Let's start first to import the data related to the spe10, grid and permeability

In [ ]:
# Define the class with the corresponding layer(s)
spe10 = Spe10(selected_layers)
# For simplicity we extract the grid form the class spe10
sd = spe10.sd

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

We set now the data, in particular the permeability tensor by specifying all its diagonal components (the off-diagonal are assumed zero) and the boundary conditions.

In [ ]:
# Permeability
perm = pp.SecondOrderTensor(
    kxx=perm_dict["kxx"], kyy=perm_dict["kyy"], kzz=perm_dict["kzz"]
)

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

# TODO define outflow and inflow type boundary conditions, right and left boundary
out_flow = 
in_flow = 


# TODO define the labels and values for the boundary faces
labels = 

bc_val = 
bc = pp.BoundaryCondition(sd, b_faces, labels)

# Collect all parameters in a dictionary
parameters = {"second_order_tensor": perm, "bc": bc, "bc_values": bc_val}
data = pp.initialize_default_data(sd, {}, key, parameters)

For simplicity, we consider the TPFA scheme to solve the problem

In [ ]:
#TODO discretize and solve with TPFA

cell_p = 

In [ ]:
mat_discr = data[pp.DISCRETIZATION_MATRICES][key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

We post process the pressure to compute the flux for each face and the projected in each cell for visualization purposes.

In [ ]:
# to export the flux
mvem = pp.MVEM(key)
mvem.discretize(sd, data)

# construct the P0 flux reconstruction
cell_q = mvem.project_flux(sd, q, data)

Finally, we export the computed solutions and the permeability field.

In [ ]:
save = pp.Exporter(sd, "sol", folder_name="ex3")

data_to_export = [
    ("kxx", np.log10(perm_dict["kxx"])),
    ("kyy", np.log10(perm_dict["kyy"])),
    ("kzz", np.log10(perm_dict["kzz"])),
    ("cell_p", cell_p),
    ("cell_q", cell_q),
]
save.write_vtu(data_to_export)